# SVM para test luminaria


## Importamos librerias

In [ ]:
import pandas as pd
import numpy as np

from typing import List 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


## Preprarar los datos y el pipeline

In [49]:
df=pd.read_csv('datasets/luminaria_con_niveles_de_riesgo 231.csv')
df.head()

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord,nivel_riesgo
0,7mo o más,No,Sí,No,"Sí, medio tiempo",Algo,No,Algunas veces,Insatisfecho/a,Algunas veces,Alto Riesgo
1,7mo o más,No,Sí,Sí,"Sí, medio tiempo",Algo,No,Siempre,Insatisfecho/a,"Sí, frecuentemente",Alto Riesgo
2,1er - 3er,Si,Sí,Sí,No,Algo,No,Algunas veces,Insatisfecho/a,Nunca,Medio Riesgo
3,1er - 3er,Si,No,Sí,"Sí, medio tiempo",Algo,No,Algunas veces,Satisfecho/a,Algunas veces,Alto Riesgo
4,7mo o más,No,No,No,"Sí, medio tiempo",No afecta/No trabajo,No,Algunas veces,Satisfecho/a,"Sí, frecuentemente",Bajo Riesgo


In [50]:
# dividimos el data set en 3 partes
# entrenamiento 70%, validacion 15% y prueba 15%

from sklearn.model_selection import train_test_split
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=42, stratify=df['nivel_riesgo'],)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['nivel_riesgo'],)
print(f'Tamaño del conjunto de entrenamiento: {len(df_train)}')
print(f'Tamaño del conjunto de validación: {len(df_val)}')
print(f'Tamaño del conjunto de prueba: {len(df_test)}')

Tamaño del conjunto de entrenamiento: 161
Tamaño del conjunto de validación: 35
Tamaño del conjunto de prueba: 35


Separamos la variable objetivo de cada set de datos

In [51]:
X_train = df_train.drop('nivel_riesgo', axis=1)
y_train = df_train['nivel_riesgo']

X_val = df_val.drop('nivel_riesgo', axis=1)
y_val = df_val['nivel_riesgo']

X_test = df_test.drop('nivel_riesgo', axis=1)
y_test = df_test['nivel_riesgo']

In [52]:
X_train.head()

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord
217,1er - 3er,Si,Sí,No,No,No afecta/No trabajo,Sí,Algunas veces,Insatisfecho/a,Nunca
104,7mo o más,Si,Sí,Sí,"Sí, medio tiempo",Algo,Sí,Algunas veces,Satisfecho/a,Algunas veces
19,7mo o más,No,Sí,No,"Sí, medio tiempo","Sí, mucho",No,Siempre,Muy satisfecho/a,Nunca
129,7mo o más,No,Sí,No,No,No afecta/No trabajo,No,Nunca,Insatisfecho/a,Nunca
84,7mo o más,No,No,No,No,No afecta/No trabajo,No,Siempre,Satisfecho/a,Algunas veces


In [53]:
X_train.columns

Index(['semestre_ord', 'desmotivacion_bin', 'considerado_abandonar_bin',
       'dificultades_economicas_bin', 'empleo_ord', 'impacto_laboral_ord',
       'reprobo_materias_bin', 'apoyo_institucional_ord',
       'satisfaccion_servicios_ord', 'actividades_extracurriculares_ord'],
      dtype='object')

### Pipeline
Necesitamos crear un pipeline que tomme el cada columna y la procese de la forma correcta
Tenemos variables binerias; como *reprobo_materia*; que con 1 y 0 basta.
Variables ordinales; como *satisfaccion_servicios_ord* que requiere valores ordinales (del 1 al 4).
Y variables categoricas; *impacto_laboral_ord*; como  que necesitan un procesado de OneHotEncoding para funcionar correctamente.

Al final se aplicara una estandarización para optimizar el funcionamiento de un modelo de maquina de soporte vectorial (SVM).

Cada variable y su propecemiento están mas detallados en [encoding_svm_bayes.json](agent/encoding_svm_bayes.json).

In [55]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

# 1. Definir las columnas según encoding_svm_bayes.json
binarias = ['dificultades_economicas_bin', 'reprobo_materias_bin']
ordinales = {
    'semestre_ord': ['1er - 3er', '4to - 6to', '7mo o más'],
    'apoyo_institucional_ord': ['Nunca', 'Algunas veces', 'Siempre'],
    'satisfaccion_servicios_ord': ['Muy insatisfecho/a', 'Insatisfecho/a', 'Satisfecho/a', 'Muy satisfecho/a'],
    'actividades_extracurriculares_ord': ['Nunca', 'Algunas veces', 'Sí, frecuentemente']
}
onehot = ['empleo_ord', 'impacto_laboral_ord']

# 2. Transformador para variables binarias
class BinarioMapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.maps = {
            'dificultades_economicas_bin': {'Sí': 1, 'No': 0},
            'reprobo_materias_bin': {'Sí': 1, 'No': 0}
        }
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_ = X.copy()
        for col, mapping in self.maps.items():
            X_[col + '_bin'] = X_[col].map(mapping)
        return X_[[col + '_bin' for col in self.maps]]

# 3. Crear el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # Binarias
        ('bin', BinarioMapper(), binarias),
        # Ordinales
        ('ord', OrdinalEncoder(categories=[ordinales[k] for k in ordinales], dtype=int),
         list(ordinales.keys())),
        # One-hot
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, dtype=int), onehot)
    ]
)

pipeline = Pipeline([
    ('pre', preprocessor),
    ('scaler', StandardScaler())
])



Verificamos la forma que nos devuelve el pipeline

In [56]:

# prubea
X_train_prep = pipeline.fit_transform(X_train)
print(f'Shape después del pipeline: {X_train_prep.shape}')


Shape después del pipeline: (161, 10)


In [57]:
# extraer nombres de columnas después del preprocesamiento
bin_cols = [f"{col}_bin" for col in binarias]
ord_cols = list(ordinales.keys())
# este es el mas importante, pues los nombres dependen de los datos
onehot_cols = pipeline.named_steps['pre'].named_transformers_['onehot'].get_feature_names_out(onehot)
all_cols = bin_cols + ord_cols + list(onehot_cols)
all_cols

['dificultades_economicas_bin_bin',
 'reprobo_materias_bin_bin',
 'semestre_ord',
 'apoyo_institucional_ord',
 'satisfaccion_servicios_ord',
 'actividades_extracurriculares_ord',
 'empleo_ord_Sí, medio tiempo',
 'empleo_ord_Sí, tiempo completo',
 'impacto_laboral_ord_No afecta/No trabajo',
 'impacto_laboral_ord_Sí, mucho']

Codificamos cada set de datos

In [58]:
X_train_encoded = pd.DataFrame(
    pipeline.transform(X_train),
    columns=all_cols,
    index=X_train.index
)

X_val_encoded = pd.DataFrame(
    pipeline.transform(X_val),
    columns=all_cols,
    index=X_val.index
)

X_test_encoded = pd.DataFrame(
    pipeline.transform(X_test),
    columns=all_cols,
    index=X_test.index
)

print(f'Cojunto de entrenamiento preprocesado: {X_train_encoded.shape}')
print(f'Cojunto de validación preprocesado: {X_val_encoded.shape}')
print(f'Cojunto de pruebas preprocesado: {X_test_encoded.shape}')

Cojunto de entrenamiento preprocesado: (161, 10)
Cojunto de validación preprocesado: (35, 10)
Cojunto de pruebas preprocesado: (35, 10)


## SVM


Creamos un primer modelo de SVM, y hacemos pruebas locales.

In [ ]:
from sklearn.svm import SVC

## SVM
svm_clf = SVC(kernel='rbf', C=1 , gamma=0.01)

svm_clf.fit(X_train_encoded, y_train)



,C,1
,kernel,'rbf'
,degree,3
,gamma,0.01
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [61]:
y_pred = svm_clf.predict(X_val_encoded)
y_pred

array(['Alto Riesgo', 'Bajo Riesgo', 'Alto Riesgo', 'Medio Riesgo',
       'Alto Riesgo', 'Alto Riesgo', 'Medio Riesgo', 'Bajo Riesgo',
       'Bajo Riesgo', 'Bajo Riesgo', 'Alto Riesgo', 'Bajo Riesgo',
       'Alto Riesgo', 'Bajo Riesgo', 'Bajo Riesgo', 'Bajo Riesgo',
       'Alto Riesgo', 'Bajo Riesgo', 'Medio Riesgo', 'Medio Riesgo',
       'Bajo Riesgo', 'Alto Riesgo', 'Bajo Riesgo', 'Medio Riesgo',
       'Alto Riesgo', 'Medio Riesgo', 'Alto Riesgo', 'Bajo Riesgo',
       'Bajo Riesgo', 'Bajo Riesgo', 'Alto Riesgo', 'Medio Riesgo',
       'Bajo Riesgo', 'Bajo Riesgo', 'Bajo Riesgo'], dtype=object)

Evaluamos el modelo.

A pesa de tener un f1 score más alto, no suele generalizar bien en otras pruebas.

In [62]:
from sklearn.metrics import f1_score
f1 = f1_score(y_val, y_pred, average='weighted')
print(f'F1-score en el conjunto de validación: {f1:.4f}')

F1-score en el conjunto de validación: 0.8509


Prueba del modelo

In [63]:
y_pred = svm_clf.predict(X_test_encoded)

f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1-score en el conjunto de validación: {f1:.4f}')

F1-score en el conjunto de validación: 0.8802


### Buesqueda aleatoria del mejor modelo
Usaremos la clase RandomizedSearchCV de skelarn para probar multiples configuraciones del modelo para encontrar el más óptimo.`

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, loguniform

param_distributions = {
    'C': randint(low=1, high=10),
    'gamma': loguniform(1e-4, 1),
    'kernel': ['rbf', 'poly']
}

svm_clf_search = SVC()

rnd_search = RandomizedSearchCV(
    svm_clf_search,
    param_distributions=param_distributions,
    n_iter=80,
    scoring='f1_weighted',
    cv=3,
    random_state=42,
    # n_jobs=-1
)

rnd_search.fit(X_train_encoded, y_train)

,estimator,SVC()
,param_distributions,"{'C': <scipy.stats....x7a8194571c10>, 'gamma': <scipy.stats....x7a819541fda0>, 'kernel': ['rbf', 'poly']}"
,n_iter,80
,scoring,'f1_weighted'
,n_jobs,None
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


Aqui tenemos los parametros que mejor se adaptaron al conunto de datos.

In [65]:
rnd_search.best_params_

{'C': 1, 'gamma': np.float64(0.0700434954523006), 'kernel': 'rbf'}

### SVM optimizado

Ahora creamos el modelo con los mejores parámetros, lo validaremos y probaremos.

In [66]:
svm_clf_optimized = SVC(
    C=9, 
    gamma=np.float64(0.0700434954523006), 
    kernel='rbf'
)

svm_clf_optimized.fit(X_train_encoded, y_train)



,C,9
,kernel,'rbf'
,degree,3
,gamma,np.float64(0.0700434954523006)
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [67]:
# validación del modelo optimizado
y_pred = svm_clf_optimized.predict(X_val_encoded)
f1 = f1_score(y_val, y_pred, average='weighted')
print(f'F1-score en el conjunto de validación: {f1:.4f}')

F1-score en el conjunto de validación: 0.8294


In [68]:
# prueba del modelo optimizado
y_pred = svm_clf_optimized.predict(X_test_encoded)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1-score en el conjunto de validación: {f1:.4f}')

F1-score en el conjunto de validación: 0.8576


El modelo parece tener un f1 score inferior en la validación, pero un valor más alto en prueba. 
El modelo se considerá que si generaliza correctamente y se aceptará para la siguiente fase de desarrollo.

## Exportar modelo

Para exportar el modelo usaremos la librería joblib, para guardar el estado de la instacia y poder acceder desde otro código.
Es importante considerar que se requerirán importar todas las librerías que se usaron en la creación del objeto en el nuevo proyecto.
Con importar con ```from sklearn.svm import SVC``` es suficiente para este modelo.

In [69]:
import joblib

joblib.dump(svm_clf_optimized, './modelos/svm_optimized.joblib')

['./modelos/svm_optimized.joblib']

In [70]:
svm_clf_optimized.get_params()

{'C': 9,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': np.float64(0.0700434954523006),
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [71]:
X_train_encoded.iloc[0].to_numpy()

array([-0.61324414,  1.79912259, -1.37171857, -0.02432881, -0.96784862,
       -1.09801174, -0.68090848, -0.27080128,  0.79136757, -0.28379026])

In [47]:
y_train.iloc[0]

'Medio Riesgo'

## Exportar el codificador

In [72]:
joblib.dump(pipeline, './modelos/svm_preprocessor.joblib')

['./modelos/svm_preprocessor.joblib']

De la misma forma que el anterior usamos la librería joblib.

**IMPORTANTE**
En un nuevo proyecto debemos importar las librerías usadas para la creación del pipeline, inlcuyendo **Librerias base, clases y modulos personalizados**

Ademas de dependencias de *sklearn* debemos importar también el código del pipeline [Pipeline](./svm_luminaria.ipynb/#pipeline), o el módulo auxiliar [pipeline_luminaria_aux.py](pipeline_luminaria_aux.py) .
